In [22]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np



In [23]:
# loading the dataframes
df = pd.read_csv("../data/CleanedFlights.csv")

dfAirports = pd.read_csv("../data/Airports.csv")

# Dropping the cancelled Flights, because they are not relevant for this visualization

df = df[df["CANCELLED"] == 0]
dfAirports.head()

C:\Users\Florian\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning:

Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.



,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [24]:
# creating dictionaries from the IATA_CODE to the Airport, Latitude and Longiture

AirportDict = pd.Series(dfAirports.AIRPORT.values, index = dfAirports.IATA_CODE).to_dict()
LatitudeDict = pd.Series(dfAirports.LATITUDE.values, index = dfAirports.IATA_CODE).to_dict()
LongitudeDict = pd.Series(dfAirports.LONGITUDE.values, index = dfAirports.IATA_CODE).to_dict()


In [25]:
# sorting the dataframe by airport and their total flights

df = df.groupby(["ORIGIN_AIRPORT", "DESTINATION_AIRPORT"]).count().sort_values("YEAR", ascending = False)

# dropping not needet columns and renaming

df.drop(df.columns[[0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]], axis = 1, inplace = True)
df.rename(columns = {df.columns[0]: "COUNT"}, inplace= True)

# adding columns for the full airport name, latitude and longitude

df.insert(column= "ORIGIN_AIRPORT_SHORT", loc = 0, value = df.index.get_level_values(0))
df.insert(column= "DESTINATION_AIRPORT_SHORT", loc = 0, value = df.index.get_level_values(1))

df.insert(column = "ORIGIN_AIRPORT_LONG", loc = 3, value = "")
df.insert(column = "ORIGIN_LATITUDE", loc = 4, value = "")
df.insert(column = "ORIGIN_LONGITUDE", loc = 5, value = "")

df.insert(column = "DESTINATION_AIRPORT_LONG", loc = 6, value = "")
df.insert(column = "DESTINATION_LATITUDE", loc = 7, value = "")
df.insert(column = "DESTINATION_LONGITUDE", loc = 8, value = "")


df["ORIGIN_AIRPORT_LONG"] = df["ORIGIN_AIRPORT_SHORT"].map(AirportDict)
df["ORIGIN_LATITUDE"] = df["ORIGIN_AIRPORT_SHORT"].map(LatitudeDict)
df["ORIGIN_LONGITUDE"] = df["ORIGIN_AIRPORT_SHORT"].map(LongitudeDict)

df["DESTINATION_AIRPORT_LONG"] = df["DESTINATION_AIRPORT_SHORT"].map(AirportDict)
df["DESTINATION_LATITUDE"] = df["DESTINATION_AIRPORT_SHORT"].map(LatitudeDict)
df["DESTINATION_LONGITUDE"] = df["DESTINATION_AIRPORT_SHORT"].map(LongitudeDict)

df = df.reset_index()
df.drop(df.columns[[0,1]], axis = 1, inplace = True)
df.head()

,DESTINATION_AIRPORT_SHORT,ORIGIN_AIRPORT_SHORT,COUNT,ORIGIN_AIRPORT_LONG,ORIGIN_LATITUDE,ORIGIN_LONGITUDE,DESTINATION_AIRPORT_LONG,DESTINATION_LATITUDE,DESTINATION_LONGITUDE
0,LAX,SFO,13406,San Francisco International Airport,37.61900,-122.37484,Los Angeles International Airport,33.94254,-118.40807
1,SFO,LAX,13115,Los Angeles International Airport,33.94254,-118.40807,San Francisco International Airport,37.61900,-122.37484
2,LAX,JFK,11875,John F. Kennedy International Airport (New Yor...,40.63975,-73.77893,Los Angeles International Airport,33.94254,-118.40807
3,JFK,LAX,11872,Los Angeles International Airport,33.94254,-118.40807,John F. Kennedy International Airport (New Yor...,40.63975,-73.77893
4,LAX,LAS,9652,McCarran International Airport,36.08036,-115.15233,Los Angeles International Airport,33.94254,-118.40807


In [27]:
# exporting the re-organized dataframe as csv to use it in an html doc
df.to_csv("DataFrameForMostCommonRoutes.csv")